To create a proof of concept for this method, we start by modelling a simple system in 2D space. Let's first assume our optical system consists of a monochromatic laser with a certain spread and wavelength, shining through a medium with a certain optical density onto a detector. The laser exists at the position $(0, 0)$, pointing towards the positive $x$ direction. The detector is positioned at $(0, p)$ where $p$ can be any positive real number. For now, the influence of the medium is neglected and the system is assumed to exist in a perfect vacuum, which has a optical density of $0$. Because this system exists in 2D space, effects of polarisation on the light is absent, as all light in 2D space is polarised.  

To simulate the light in 2 dimensions, this [blog](https://www.dev-mind.blog/simulating-light/) and this [article](https://static.uni-graz.at/fileadmin/_Persoenliche_Webseite/puschnig_peter/unigrazform/Theses/Ohner_FDTD_Bachelorarbeit_final.pdf) will be used as a guide for implementing the Maxwell equations. 

In [117]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2

speed_of_light = 3E8

permitivity = 0.4
permeability = 0.05
conductivity = 0.01

delta_x = 0.01
delta_y = delta_x
delta_time = 0.001 #10 / (np.sqrt(2) * speed_of_light)

nt = 20000
nx = 256
ny = 256

cex = permitivity * delta_time / delta_x
cey = permitivity * delta_time / delta_y
chx = permeability * delta_time / delta_x
chy = permeability * delta_time / delta_y

ex = np.zeros((nx, ny + 1))
ey = np.zeros((nx + 1, ny))
hz = np.zeros((nx, ny))

sourcepoint = (int(nx / 2), int(ny / 2))

nframes = 1200
out = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*'MJPG'), 60, (nx, ny))
    
for t in range(nt):
    ex[:,1:-1] = ex[:,1:-1] + cex * (hz[:,1:] - hz[:,:-1])
    ey[1:-1,:] = ey[1:-1,:] - cey * (hz[1:,:] - hz[:-1,:])
    hz = hz - chx * (ey[1:,:] - ey[:-1,:]) + chy * (ex[:,1:] - ex[:,:-1])
    hz[sourcepoint] = np.sin(t / 500)
    
    if t % int(nt / nframes) == 0:
        img = np.stack((np.clip(255 * (hz + 1) / 2, 0, 255).astype(np.uint8), np.zeros(hz.shape, dtype=np.uint8), np.zeros(hz.shape, dtype=np.uint8)), axis=2)
        out.write(img)
    
out.release()